In [1]:
import numpy as np
import pandas as pd

In [2]:
#read train and test data

In [3]:
train_df = pd.read_csv('./Data/train.csv')
test_df = pd.read_csv('./Data/test.csv')
print(train_df.shape)
print(test_df.shape)

(159571, 8)
(153164, 2)


In [4]:
FASTTEXT_EMBEDFILE = './fasttext_wordvec/wiki.en.vec'

In [5]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
train_df = train_df.dropna(subset=['comment_text'])

In [7]:
X_train = train_df['comment_text'].values
label_column_names = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y_train = train_df[label_column_names].values
X_test = test_df['comment_text'].values

In [8]:
#preprocess the text

In [9]:
from keras.preprocessing import text,sequence

/Users/mozhiwen/anaconda3/envs/nlp/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
vocab_size = 30000
max_len = 100
embed_size = 300

In [11]:
def text2sequence(vocab_size,max_len,X_train,X_test):
    tokenizer = text.Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(list(X_train)+list(X_test))
    word_index = tokenizer.word_index
    X_train = sequence.pad_sequences(tokenizer.texts_to_sequences(X_train),maxlen=max_len)
    X_test = sequence.pad_sequences(tokenizer.texts_to_sequences(X_test),maxlen=max_len)
    return X_train, X_test , word_index

In [12]:
X_train, X_test, word_index = text2sequence(vocab_size,max_len,X_train,X_test)

In [32]:
with open(FASTTEXT_EMBEDFILE,'r') as f:
    fasttext_wordvec = {}
    vs, ems = f.readline()[:-1].split()
    print('In this embedding file,\n vocab size: %s,\n embed size: %s' % (vs,ems))
    line = f.readline()
    while line:
        key_v_list = line[:-1].rstrip().rsplit(' ',300)
        if len(key_v_list[1:]) != 300:
            raise ValueError('embed size is not 300:',key_v_list)
        try:
            fasttext_wordvec[key_v_list[0]] = np.asarray(key_v_list[1:],dtype='float32')
        except:
            raise ValueError(key_v_list)
        line = f.readline()

In this embedding file,
 vocab size: 2519370,
 embed size: 300


In [35]:
np.random.seed(42)

In [36]:
vocab_size = min(vocab_size,len(word_index))

In [38]:
embedding_matrix = np.random.uniform(-1,1,(vocab_size,embed_size))

In [39]:
for word, i in word_index.items():
    if i > vocab_size:break
    if word in fasttext_wordvec:
        embedding_matrix[i-1] = fasttext_wordvec[word]
    else:
        continue

In [46]:
'''embed_mat_file = './Data/embed_mat.npy'
embedding_matrix.dump(embed_mat_file)'''

In [47]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D